# Test NIP-11 Relay Information Document

Interactive notebook for testing the `Nip11` model:
- Fetch from real relays (clearnet and overlay networks)
- Data access via `metadata.data["key"]`
- Convenience properties for common fields
- Proxy support for Tor/I2P/Loki relays
- Parsing and validation
- Error handling

In [ ]:
import sys
sys.path.insert(0, "../src")

import asyncio
import json
from models.nip11 import Nip11, Nip11FetchError
from models.relay import Relay
from models.metadata import Metadata

print("Imports loaded successfully")

## 1. Basic Fetch from Clearnet Relay

In [ ]:
# Test fetch from a well-known relay
relay = Relay("wss://relay.damus.io")
print(f"Relay: {relay.url}")
print(f"Network: {relay.network}")
print(f"Scheme: {relay.scheme}")
print(f"Host: {relay.host}")

In [ ]:
# Fetch NIP-11 (no proxy needed for clearnet)
nip11 = await Nip11.fetch(relay)
print(f"Fetch successful: {nip11 is not None}")
print(f"Type: {type(nip11).__name__}")

## 2. Convenience Properties

In [ ]:
# Access common fields via convenience properties
print("=== Convenience Properties ===")
print(f"name: {nip11.name}")
print(f"supported_nips: {nip11.supported_nips}")
print(f"tags: {nip11.tags}")

## 3. Data Access via metadata.data

In [ ]:
# Access data via metadata.data dict
data = nip11.metadata.data

print("=== Base Fields ===")
print(f"name: {data['name']}")
print(f"description: {data['description']}")
print(f"pubkey: {data['pubkey']}")
print(f"contact: {data['contact']}")
print(f"software: {data['software']}")
print(f"version: {data['version']}")

In [ ]:
# List fields
print("=== List Fields ===")
print(f"supported_nips: {data['supported_nips']}")
print(f"relay_countries: {data['relay_countries']}")
print(f"language_tags: {data['language_tags']}")
print(f"tags: {data['tags']}")

## 4. Limitation Object

In [ ]:
# Limitation dict (via convenience property)
print("=== Limitation ===")
limitation = nip11.limitation

if limitation and any(v is not None for v in limitation.values()):
    print(f"  max_message_length: {limitation.get('max_message_length')}")
    print(f"  max_subscriptions: {limitation.get('max_subscriptions')}")
    print(f"  max_limit: {limitation.get('max_limit')}")
    print(f"  auth_required: {limitation.get('auth_required')}")
    print(f"  payment_required: {limitation.get('payment_required')}")
else:
    print("  No limitations defined")

## 5. Retention Policies

In [ ]:
# Retention policies (via convenience property)
print("=== Retention Policies ===")
retention = nip11.retention

if retention:
    for i, entry in enumerate(retention):
        print(f"\n  [{i}] kinds: {entry.get('kinds')}")
        print(f"      time:  {entry.get('time')} seconds")
        print(f"      count: {entry.get('count')}")
else:
    print("  No retention policies defined")

## 6. Fee Schedules

In [ ]:
# Fees
print("=== Fees ===")
fees = data.get('fees', {})

if fees and any(v is not None for v in fees.values()):
    for category in ('admission', 'subscription', 'publication'):
        fee_list = fees.get(category)
        if fee_list:
            print(f"\n  {category.upper()}:")
            for entry in fee_list:
                amount = entry.get('amount', '?')
                unit = entry.get('unit', '?')
                period = entry.get('period')
                kinds = entry.get('kinds')
                line = f"    {amount} {unit}"
                if period:
                    line += f" / {period}s"
                if kinds:
                    line += f" (kinds: {kinds})"
                print(line)
else:
    print("  No fees defined")

## 7. Full JSON Output

In [ ]:
# All schema keys are present (with None for missing)
print("=== All Schema Keys ===")
print(f"Keys: {list(nip11.metadata.data.keys())}")
print()
print(json.dumps(nip11.metadata.data, indent=2, default=str))

## 8. Conversion to RelayMetadata

In [ ]:
# Convert to RelayMetadata for DB storage
relay_metadata = nip11.to_relay_metadata()
print(f"RelayMetadata type: {type(relay_metadata).__name__}")
print(f"metadata_type: {relay_metadata.metadata_type}")
print(f"generated_at: {relay_metadata.generated_at}")
print(f"relay url: {relay_metadata.relay.url}")

## 9. Fetch Multiple Relays

In [ ]:
# Test multiple relays in parallel
relay_urls = [
    "wss://relay.damus.io",
    "wss://nos.lol",
    "wss://relay.nostr.band",
    "wss://nostr.wine",
    "wss://relay.snort.social",
]

async def fetch_nip11_safe(url: str) -> tuple[str, Nip11 | None, Exception | None]:
    relay = Relay(url)
    try:
        nip11 = await Nip11.fetch(relay, timeout=5.0)
        return url, nip11, None
    except Exception as e:
        return url, None, e

results = await asyncio.gather(*[fetch_nip11_safe(url) for url in relay_urls])

print("=== Multiple Relay Results ===")
for url, nip11_result, error in results:
    if nip11_result:
        nips = nip11_result.supported_nips
        nips_count = len(nips) if nips else 0
        print(f"✓ {url}: {nip11_result.name or 'No name'} - {nips_count} NIPs")
    else:
        print(f"✗ {url}: {type(error).__name__}")

## 10. Fetch with Proxy (Tor Relay)

To test Tor relays, you need a running Tor proxy (SOCKS5 on port 9050).

In [ ]:
# Example: Fetch from a Tor relay (requires Tor proxy running)
# Uncomment to test if you have Tor running locally

# tor_relay = Relay("ws://oxtrdevav64z64yb7x6rjg4ntzqjhedm5b5zjqulugknhzr46ny2qbad.onion")
# tor_proxy = "socks5://127.0.0.1:9050"
# 
# try:
#     nip11_tor = await Nip11.fetch(tor_relay, proxy_url=tor_proxy, timeout=30.0)
#     print(f"Tor relay name: {nip11_tor.name}")
#     print(f"Software: {nip11_tor.metadata.data.get('software')}")
# except Nip11FetchError as e:
#     print(f"Tor fetch failed: {e.cause}")

print("Tor test skipped (uncomment to run with local Tor proxy)")

## 11. Proxy Behavior

The `proxy_url` parameter behavior:
- **If provided**: Uses proxy regardless of relay network type
- **If not provided + clearnet relay**: Direct connection (works)
- **If not provided + overlay relay**: Fails (DNS can't resolve .onion/.i2p/.loki)

In [ ]:
# Demonstrate: overlay relay without proxy (will fail)
try:
    tor_relay = Relay("ws://example.onion")
    result = await Nip11.fetch(tor_relay, timeout=3.0)
    print("Unexpected success!")
except Nip11FetchError as e:
    print(f"Expected failure for overlay without proxy:")
    print(f"  Cause: {type(e.cause).__name__}")

## 12. Test Parsing with Synthetic Data

In [ ]:
# Create Nip11 from synthetic data
test_relay = Relay("wss://test.relay.example")

synthetic_data = {
    "name": "Test Relay",
    "description": "A test relay for parsing",
    "supported_nips": [1, 2, 4, 9, 11, 40],
    "limitation": {
        "max_message_length": 128000,
        "auth_required": True,
    },
    "fees": {
        "admission": [{"amount": 21000, "unit": "sats"}],
    },
}

nip11_synthetic = Nip11(relay=test_relay, metadata=Metadata(synthetic_data))

print("=== Synthetic Nip11 ===")
print(f"name: {nip11_synthetic.name}")
print(f"supported_nips: {nip11_synthetic.supported_nips}")
print(f"limitation: {nip11_synthetic.limitation}")
print(f"fees: {nip11_synthetic.metadata.data.get('fees')}")

## 13. Parsing Edge Cases

In [ ]:
# Test: Invalid types are filtered out
invalid_data = {
    "name": 12345,  # Should be string -> None
    "description": "Valid description",
    "supported_nips": [1, 2, "three", 4],  # Non-ints filtered
}

nip11_invalid = Nip11(relay=test_relay, metadata=Metadata(invalid_data))

print("=== Invalid Types Filtered ===")
print(f"name: {nip11_invalid.name} (was int -> None)")
print(f"description: {nip11_invalid.metadata.data.get('description')}")
print(f"supported_nips: {nip11_invalid.supported_nips} (filtered to [1, 2, 4])")

In [ ]:
# Test: Empty iterables become None
empty_data = {
    "name": "Empty Test",
    "supported_nips": [],  # Empty list -> None
    "limitation": {},  # Empty dict -> all keys with None values
}

nip11_empty = Nip11(relay=test_relay, metadata=Metadata(empty_data))

print("=== Empty Iterables ===")
print(f"supported_nips: {nip11_empty.supported_nips} (was [] -> None)")
print(f"limitation: Has all keys with None values")

## 14. Error Handling

In [ ]:
# Test: Invalid relay URL
invalid_relay = Relay("wss://nonexistent.relay.invalid")

try:
    result = await Nip11.fetch(invalid_relay, timeout=3.0)
except Nip11FetchError as e:
    print("=== Nip11FetchError ===")
    print(f"relay: {e.relay.url}")
    print(f"cause: {type(e.cause).__name__}")

In [ ]:
# Test: Empty metadata raises ValueError
try:
    nip11_empty_fail = Nip11(relay=test_relay, metadata=Metadata({}))
except ValueError as e:
    print("=== Empty Metadata ===")
    print(f"Correctly raised: {e}")

## 15. Class Defaults

In [ ]:
print("=== Class Defaults ===")
print(f"_FETCH_TIMEOUT: {Nip11._FETCH_TIMEOUT} seconds")
print(f"_FETCH_MAX_SIZE: {Nip11._FETCH_MAX_SIZE} bytes ({Nip11._FETCH_MAX_SIZE // 1024} KB)")

## Done!

### Key Points:
- Access fields via `nip11.metadata.data["key"]`
- Convenience properties: `name`, `supported_nips`, `limitation`, `retention`, `tags`
- All schema keys are present (with `None` for missing)
- Invalid types are silently converted to `None`
- Empty iterables become `None`
- Use `proxy_url` for Tor/I2P/Loki relays
- SSL fallback: clearnet relays with invalid certs are retried with `allow_insecure=True`